In [1]:
import higlass as hg
import cooler
import glob

In [2]:
coords = "chr1:44,000,000-68,000,000"
coolers = list(sorted(glob.glob("../../data/output/nfcore_hic/mcools/hg38_00?_MCF10A*.mcool")))

tads2d_10k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_10000_2D.beddb"))
)
tads2d_20k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_20000_2D.beddb"))
)
tads2d_50k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_50000_2D.beddb"))
)
tads2d_100k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_100000_2D.beddb"))
)

ins_10k = list(sorted(glob.glob("../../data/output/tad_analysis/ICE/10000/hg38_00?_MCF10A*.bw")))
ins_20k = list(sorted(glob.glob("../../data/output/tad_analysis/ICE/20000/hg38_00?_MCF10A*.bw")))
ins_50k = list(sorted(glob.glob("../../data/output/tad_analysis/ICE/50000/hg38_00?_MCF10A*.bw")))
ins_100k = list(sorted(glob.glob("../../data/output/tad_analysis/ICE/100000/hg38_00?_MCF10A*.bw")))

In [3]:
def compute_domain(coords, clr: cooler.Cooler, padding=0.0):
    df = clr.matrix(as_pixels=True).fetch(coords)

    pos1 = df["bin1_id"].min() * clr.binsize
    pos2 = df["bin1_id"].max() * clr.binsize

    if padding != 0.0:
        extent = pos2 - pos1
        new_extent = int(padding * extent)
        pos1 -= (new_extent - extent) // 2
        pos2 += (new_extent - extent) // 2

    return max(0, pos1), pos2

## TADs as 2D domains

In [4]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

cooler_tiles = {}
tad_tiles = {}
cooler_tracks = {}

for bedpe, clr in zip(tads2d_50k, coolers):
    cooler_tiles[clr] = hg.cooler(clr)
    tad_tiles[clr] = hg.bed2ddb(bedpe)
    cooler_tracks[clr] = hg.combine(
        cooler_tiles[clr].track("linear-heatmap"),
        tad_tiles[clr].track("linear-2d-rectangle-domains"),
        height=70,
    )


chrom_sizes_track = list(cooler_tiles.values())[0].track("chromosome-labels")
ga_track = gene_annotation_tile.track("horizontal-gene-annotations")
top_track = hg.combine(ga_track, chrom_sizes_track, height=150)

uri = coolers[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"
domain = compute_domain(coords, cooler.Cooler(uri))

tracks = [(top_track, "top")]

for key, track in cooler_tracks.items():
    tracks.append((track, "top"))

view = hg.view(*tracks).domain(x=domain, y=domain)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[CombinedTrack(type='combined', uid='5ee59d72-e309-4dbc-8d39-7b94dcdd95f6', width=None, height=150, options=None, contents=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='cb9b5bd8-71a5-411f-8781-d2d64ee22a9b', width=None, height=None, options={'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='chromosome-labels', uid='83d21695-cb7f-4ea2-9a6e-d5c6bf21a5b1', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='be1b011f-bf25-46f8-8913-b71960b2e6cc', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='04f810bc-e8b2-42ae-9fee-16fa62f49c53', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='37b59a41512dd7b29a8e23ee9faa94fe', server='http://localhost:40679/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='3d246b50-ba22-4a90-ab6b-d761f9de2a80', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='bff4ac53-88cd-4abb-87b3-d1b4eaff7e6a', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='ad01de40dfecfd01d435287c167bd33b', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='a187c19f-4e49-4ccc-bc66-33ab0955789d', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d99d8524e24840c79a514cbddd5b477b', server='http://localhost:40679/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='f8a97d07-0237-4693-b9bc-1c0f36a7f438', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='e7691230-e0aa-44dc-a398-2d3006570424', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='335f43c2893812a8f8d92ac88375b18f', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='867ca10b-7cd3-403c-aabe-1e80d18a7eda', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='7837de292fddf348e6808d9f8a7be466', server='http://localhost:40679/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='f82c2b55-16b8-4675-9861-ec35ce4e1ba7', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='24ac4437-4eb1-4e7b-90ee-163e920b8d20', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='4a8de3b966951cb85def4c4d9c1d2d7a', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='32546302-e733-4195-9671-31d69b671ba5', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='3bf166c7be1edd41440cf7a8b608b4c9', server='http://localhost:40679/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='f03b7aa6-535e-4c5b-bb55-b97a9c86e351', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='4d83348b-e7a1-4655-ae56-2a95b91b52ed', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='1b9842a1ee691f500e75a7134e91497e', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='b5143b48-b0cd-4133-a0bb-0fd8c0a283b2', width=None, height=None, opti

## 50 kbp TADs as BED tracks

In [5]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

cooler_tiles = {}
tad_tiles = {}
cooler_tracks = {}
tad_tracks = {}

for bed50k, clr in zip(tads2d_50k, coolers):
    cooler_tiles[clr] = hg.cooler(clr)

    cooler_tracks[clr] = cooler_tiles[clr].track("linear-heatmap", height=75)
    tad_tracks[clr] = [hg.bed2ddb(bed).track("bedlike", height=20).opts(annotationStyle="segment") for bed in (bed50k,)]

chrom_sizes_track = list(cooler_tiles.values())[0].track("chromosome-labels")

uri = coolers[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"
domain = compute_domain(coords, cooler.Cooler(uri))

views = {}

tracks = [(chrom_sizes_track, "top")]

for tad_tracks, clr_track in zip(tad_tracks.values(), cooler_tracks.values()):
    tracks.append((clr_track, "top"))
    for trk in tad_tracks:
        tracks.append((trk, "top"))

view = hg.view(*tracks).domain(x=domain, y=domain)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='chromosome-labels', uid='55603184-61d1-4509-9f59-da7776e40a73', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='ee815028-7857-4687-ab48-db068498ce3f', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='37b59a41512dd7b29a8e23ee9faa94fe', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='086f2066-0dcc-49d0-9d76-1a6e7efed909', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='ad01de40dfecfd01d435287c167bd33b', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='8b59a918-5d63-4885-aa02-6cadc9954d6a', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d99d8524e24840c79a514cbddd5b477b', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='907d6f11-672e-4a3f-81b8-a8d29ad0714c', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='335f43c2893812a8f8d92ac88375b18f', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='146e3dfc-914d-4c8e-ad43-bc1caa2b7cb8', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='7837de292fddf348e6808d9f8a7be466', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='73dad973-9d6b-473f-a12c-e76936cb389e', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='4a8de3b966951cb85def4c4d9c1d2d7a', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='2e5774f6-6d4c-4387-bf28-181951a0d122', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='3bf166c7be1edd41440cf7a8b608b4c9', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='45df7709-7e9a-4103-9e13-a1fa9378e6f2', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='1b9842a1ee691f500e75a7134e91497e', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='de239333-b678-4723-817b-645b3f9442eb', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='2fe00402c47abf1d38ac3b726730ed6a', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='91775925-672c-41b8-961f-5952c61882b8', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='249786704855055fb4f35174ca507bf2', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='69f17873-259f-4aff-9b0e-fe6ab5aa5143', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='76db855a2374c5753a75d5a179166c30', server='http://localhost:40679/tilesets/api/v1/', type='bedlike', uid='4f04dfd9-efaf-43f7-a83e-0247859fbe9c', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='61d0cfe7-900c-4580-aaad-a1a52aa7104a', autocompleteSource=None, chromInfoPath=None, genomePositionSearchB

In [6]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

cooler_tiles = {}
tad_tiles = {}
cooler_tracks = {}
tad_tracks = {}

for bw50k, clr in zip(ins_50k, coolers):
    cooler_tiles[clr] = hg.cooler(clr)

    cooler_tracks[clr] = cooler_tiles[clr].track("linear-heatmap", height=75)
    tad_tracks[clr] = [hg.bigwig(bw).track("line", height=50) for bw in (bw50k,)]

chrom_sizes_track = list(cooler_tiles.values())[0].track("chromosome-labels")

uri = coolers[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"
domain = compute_domain(coords, cooler.Cooler(uri))

views = {}

tracks = [(chrom_sizes_track, "top")]

for tad_tracks, clr_track in zip(tad_tracks.values(), cooler_tracks.values()):
    tracks.append((clr_track, "top"))
    for trk in tad_tracks:
        tracks.append((trk, "top"))

view = hg.view(*tracks).domain(x=domain, y=domain)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='chromosome-labels', uid='9bb70e77-eca1-410d-8c63-fe30da170f8c', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='cd97752c-6509-4b21-ae64-937028e415ab', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='bc99e416c2a6401ad54a56e11b30ca0b', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='b0e81640-8633-40f5-9887-8c45e102c77d', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='ad01de40dfecfd01d435287c167bd33b', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='783cce59-bce4-4a3b-a586-2900edcab4f9', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='a04a71d2de8b69fc03fd0d3131fdd4c6', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='c6ea3d87-614b-4e8b-a48f-6d52288c54d7', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='335f43c2893812a8f8d92ac88375b18f', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='11852a2f-f55c-436a-9c78-9808367f2b65', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d4215dc07c85a20843e9955e58b83ef8', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='98af6ae8-88ac-4805-95b4-7fdf314792a2', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='4a8de3b966951cb85def4c4d9c1d2d7a', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='6c3d3034-cccb-49e3-a270-dd27a2d164a3', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d1baa0b49ffe0b92d338f28b95c27abb', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='36a65e1d-0d02-45f2-9a08-585ee166aac8', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='1b9842a1ee691f500e75a7134e91497e', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='76c0210f-9b4d-4482-951c-d1fd802ad091', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='fe74205cdd6ace4eb3b74e0de542e9ca', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='75c61b99-09a2-4d4e-8acc-31344bb5a0aa', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='249786704855055fb4f35174ca507bf2', server='http://localhost:40679/tilesets/api/v1/', type='linear-heatmap', uid='d1d39972-d336-4dd4-b2c4-e91f4278813a', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='f3f55868a802db9357824185a4f8826a', server='http://localhost:40679/tilesets/api/v1/', type='line', uid='3ee935ad-e135-4e57-bec0-8cc6c2ec028b', width=None, height=50, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='33161c91-abd2-4967-b08a-adaf6639175f', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(44000000.0, 67999000.0), initialYDomain=(44000000.0, 67999000.0), overlays=None, selectionView=None, zoomFixed=N